In [1]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

import numpy as np
import pandas as pd

import string
import nltk
import glob
import re

In [2]:
path = "./data/files"

file_list = []

for file in glob.glob(path + "/*.xlsx"):
    file_list.append(pd.read_excel(file))

In [3]:
df_slang1 = pd.read_csv("slang/kamus_belanja.csv")
df_slang2 = pd.read_csv("slang/slang_indo.csv")

In [4]:
df = pd.concat(file_list, ignore_index=True)
df.head()

,Index,Text,Lang,Account,Date,URL,Anger,Disgusted,Afraid,Happy,...,Keywords Disgusted,Keywords Afraid,Keywords Happy,Keywords Sad,Keywords Shocked,Notes 1,Notes 2,Foldername,Filename,PIC
0,0,RT @Heraloebss: 16/9/2021 \nAmbruknya SDN Di w...,NaN,BossTemlen,2021-09-17 03:36:33,https://www.twitter.com/BossTemlen/status/1438...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210917_130822_emotions_1.xlsx,NaN
1,1,"RT @nitarose38: Bismillah, semoga lancar https...",NaN,BossTemlen,2021-09-17 03:28:58,https://www.twitter.com/BossTemlen/status/1438...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210917_130822_emotions_1.xlsx,NaN
2,2,️,NaN,BossTemlen,2021-09-17 03:21:04,https://www.twitter.com/BossTemlen/status/1438...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210917_130822_emotions_1.xlsx,NaN
3,3,JELAS SEKALI ada yang salah dalam sistem berne...,NaN,BossTemlen,2021-09-17 00:32:00,https://www.twitter.com/BossTemlen/status/1438...,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210917_130822_emotions_1.xlsx,NaN
4,4,PANCASILA\n.....\n5. Keadilan sosial bagi selu...,NaN,BossTemlen,2021-09-17 00:22:31,https://www.twitter.com/BossTemlen/status/1438...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210917_130822_emotions_1.xlsx,NaN


In [5]:
columns = ['Text', 'Anger', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Shocked']

df = df[columns]
df.head()

,Text,Anger,Disgusted,Afraid,Happy,Sad,Shocked
0,RT @Heraloebss: 16/9/2021 \nAmbruknya SDN Di w...,NaN,NaN,NaN,NaN,NaN,NaN
1,"RT @nitarose38: Bismillah, semoga lancar https...",NaN,NaN,NaN,NaN,NaN,NaN
2,️,NaN,NaN,NaN,NaN,NaN,NaN
3,JELAS SEKALI ada yang salah dalam sistem berne...,1.0,NaN,NaN,NaN,NaN,NaN
4,PANCASILA\n.....\n5. Keadilan sosial bagi selu...,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(40000, 7)

In [7]:
df[['Anger', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Shocked']] = df[['Anger', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Shocked']].fillna(0).astype('int64')
df.head()

,Text,Anger,Disgusted,Afraid,Happy,Sad,Shocked
0,RT @Heraloebss: 16/9/2021 \nAmbruknya SDN Di w...,0,0,0,0,0,0
1,"RT @nitarose38: Bismillah, semoga lancar https...",0,0,0,0,0,0
2,️,0,0,0,0,0,0
3,JELAS SEKALI ada yang salah dalam sistem berne...,1,0,0,0,0,0
4,PANCASILA\n.....\n5. Keadilan sosial bagi selu...,0,0,0,0,0,0


In [8]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TWEETS

#Clean emojis from text
def strip_emoji(text):
    return re.sub(r'\d+(.*?)[\u263a-\U0001f645]', r" ", text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', ' ').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", " ", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r' ', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans(' ', ' ', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append(' ')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

def drop_small_text(text):
    text = re.sub(r'RT', " ", text)
    return np.nan if len(text) < 20 else text

df['Text'] = df['Text'].apply(lambda x: drop_small_text(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(x)))))))
df.dropna(inplace=True)
df.sample(10)

,Text,Anger,Disgusted,Afraid,Happy,Sad,Shocked
33361,rt damnnnn i gonna do thisssss to my carrr,0,0,0,0,0,0
28655,rt jika bkenan bacalah baik2yg sayalingkari merah,0,0,0,0,0,0
27160,di rejim ini bkn sj ngr ancur tp jg pjbt nya p...,0,0,0,0,0,0
32268,anjg itu tulisan bold bener ampe puyeng liatnya,0,0,0,0,0,0
6756,viral video drakor terbaru,0,0,0,0,0,0
28245,kalau jkw biasa dan sengaja masuk got kalau wa...,0,0,0,0,0,0
20411,emang siapa yg mau milih ya kecuali yg ga waras,0,0,0,0,0,0
20420,lurah sudirman tapi jalan ke sumedang mau ke m...,0,0,0,0,0,0
30711,rt si pro new yg sdh follow bantu mksh,0,0,0,0,0,0
26069,alhamdulillah orang sombong itu akhirnya akan ...,0,0,0,0,0,0


In [9]:
df.to_csv("data/data.csv", index=False)